### Checking out data

In [1]:
import pathlib
 
adam_path = pathlib.Path("/home/dhaval/adam_data/")
aneurysm_patients = 0

for path in sorted(adam_path.rglob("location.txt")):
    data = open(str(path), "r").readlines()
    parent = path.parent.parts[-1]
    
    print("Patient: ", str(parent), "Location: ", data)
    
    if len(data) > 0:
        aneurysm_patients += 1

Patient:  10001 Location:  []
Patient:  10002 Location:  []
Patient:  10003 Location:  []
Patient:  10004 Location:  []
Patient:  10006 Location:  []
Patient:  10009 Location:  []
Patient:  10010 Location:  []
Patient:  10011 Location:  []
Patient:  10014 Location:  []
Patient:  10015 Location:  []
Patient:  10016 Location:  []
Patient:  10017 Location:  []
Patient:  10018 Location:  []
Patient:  10019 Location:  []
Patient:  10020 Location:  []
Patient:  10021 Location:  ['377, 221, 89, 3.0']
Patient:  10024 Location:  ['314, 244, 41, 4.16\n', '234, 237, 42, 2.05']
Patient:  10026 Location:  ['470, 396, 75, 3.99']
Patient:  10027 Location:  ['252, 236, 37, 2.1']
Patient:  10029 Location:  ['341, 216, 27, 4.12']
Patient:  10030 Location:  ['303, 213, 65, 3.84\n', '208, 224, 64, 0.7']
Patient:  10031 Location:  ['258, 207, 41, 3.56']
Patient:  10032 Location:  ['217, 223, 67, 1.27\n', '241, 257, 77, 0.75']
Patient:  10033 Location:  ['293, 223, 40, 4.14']
Patient:  10034 Location:  ['28

In [ ]:
print(aneurysm_patients)

In [ ]:
import random

# Full adam train dataset: 
# 35B and 35F of same subject with aneurysm
# 23 unique with aneurysm
# 20 without aneurysm
# 113 cases total

# Train   and val split: 
# 26B-26F and 9B-9F
# 17      and 6
# 15      and 5

without = range(10001, 10021)
unique = range(10021, 10044)
same = range(10044, 10079)

random.seed(8)

val_without = sorted(random.sample(without, k=5))
print(val_without)
    
val_unique = sorted(random.sample(unique, k=6))
print(val_unique)

val_same = sorted(random.sample(same, k=9))
val_same_double = []

for i in val_same:
    val_same_double.append(str(i) + 'B')
    val_same_double.append(str(i) + 'F')


In [ ]:
import shutil

train_adam_path = adam_path.joinpath("train/")
val_adam_path = adam_path.joinpath("val/")

for path in sorted(adam_path.glob("*/")):
    print(path)
    patient = path.parts[-1]
    
    if patient in str(val_without) or patient in str(val_unique) or patient in val_same_double:
        shutil.move(str(path), str(val_adam_path))

    elif patient != "train" and patient != "val":
        print(patient)
        shutil.move(str(path), str(train_adam_path))
    

In [4]:
import nibabel as nib
import niwidgets as nw

case_num = 10012

MRA_path = str(adam_path.joinpath("val/", str(case_num), "orig/TOF.nii.gz"))
print(MRA_path)

MRA_affine = nib.load(MRA_path).affine

MRA_widget = nw.NiftiWidget(MRA_path)
MRA_widget.nifti_plotter()

/home/dhaval/adam_data/val/10012/orig/TOF.nii.gz


<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=255, continuous_update=False, description='x', max=511), IntSlider(value…

In [15]:
from data_utils import MRAData

train_data = MRAData(adam_path, patch_size=[124, 124, 60], mode="train")

MRA_path = train_data.cases_dirs[32]

raw_path = MRA_path.joinpath("pre/TOF.nii.gz")
seg_path = MRA_path.joinpath("aneurysms.nii.gz")

location_path = MRA_path.joinpath("location.txt")

# Get center of aneurysm from location.txt file of case
aneurysm_location_coords = train_data.get_aneurysm_coords(location_path)

print("Location: ", aneurysm_location_coords)

raw_affine = nib.load(raw_path).affine

print(raw_path)
MRA_widget = nw.NiftiWidget(raw_path)
MRA_widget.nifti_plotter()

print(seg_path)
seg_widget = nw.NiftiWidget(seg_path)
seg_widget.nifti_plotter()

save_name = MRA_path.joinpath("patch.nii.gz")
output = train_data[32][0].squeeze(0)
print(output.size())
output = output.detach().numpy()
print(output.shape)
out_img = nib.Nifti1Image(output, raw_affine)
nib.save(out_img, save_name)

print(save_name)
test_widget = nw.NiftiWidget(save_name)
test_widget.nifti_plotter()

Location:  [[221.0, 220.0, 35.0, 2.1], [253.0, 267.0, 40.0, 2.03]]
/home/dhaval/adam_data/train/10044B/pre/TOF.nii.gz


<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=255, continuous_update=False, description='x', max=511), IntSlider(value…

/home/dhaval/adam_data/train/10044B/aneurysms.nii.gz


<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=255, continuous_update=False, description='x', max=511), IntSlider(value…

torch.Size([124, 124, 60])
(124, 124, 60)
/home/dhaval/adam_data/train/10044B/patch.nii.gz


<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=61, continuous_update=False, description='x', max=123), IntSlider(value=…